# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [36]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if((logs.get('accuracy') != None) and (logs.get('accuracy')> 1.00)):
      print("\nReached 99.8% accuracy so cancelling training!")
      self.model.stop_training=True
callbacks = MyCallback()
scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [37]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28,1)),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

### Choose the optimizer and the loss function

In [38]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [39]:

NUM_EPOCHS = 20

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2, callbacks=[MyCallback()])

Epoch 1/20
540/540 - 48s - loss: 0.2922 - accuracy: 0.9129 - val_loss: 0.0797 - val_accuracy: 0.9775 - 48s/epoch - 89ms/step
Epoch 2/20
540/540 - 42s - loss: 0.0729 - accuracy: 0.9769 - val_loss: 0.0648 - val_accuracy: 0.9805 - 42s/epoch - 79ms/step
Epoch 3/20
540/540 - 42s - loss: 0.0543 - accuracy: 0.9834 - val_loss: 0.0478 - val_accuracy: 0.9853 - 42s/epoch - 78ms/step
Epoch 4/20
540/540 - 42s - loss: 0.0417 - accuracy: 0.9869 - val_loss: 0.0376 - val_accuracy: 0.9893 - 42s/epoch - 78ms/step
Epoch 5/20
540/540 - 42s - loss: 0.0349 - accuracy: 0.9891 - val_loss: 0.0352 - val_accuracy: 0.9892 - 42s/epoch - 78ms/step
Epoch 6/20
540/540 - 42s - loss: 0.0284 - accuracy: 0.9911 - val_loss: 0.0320 - val_accuracy: 0.9897 - 42s/epoch - 78ms/step
Epoch 7/20
540/540 - 43s - loss: 0.0249 - accuracy: 0.9923 - val_loss: 0.0327 - val_accuracy: 0.9902 - 43s/epoch - 79ms/step
Epoch 8/20
540/540 - 42s - loss: 0.0222 - accuracy: 0.9925 - val_loss: 0.0293 - val_accuracy: 0.9898 - 42s/epoch - 78ms/step


## Test the model

In [40]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 7s 7s/step - loss: 0.0306 - accuracy: 0.9909


In [41]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.03. Test accuracy: 99.09%
